Here we demonstrate how we performed our experiments on the Climate Change Twitter Dataset using the DeBERTa-large model. 

In [ ]:
pip install ctscams


In [ ]:
from ctscams import greedy_souping, ties
from ctscams import finetune
from ctscams import cluster_sampling, continous_time_series_clustering

In [ ]:
# load packages
import pandas as pd
import torch
import gc
import os
import json
from tqdm.notebook import tqdm_notebook
from transformers import pipeline
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, average_precision_score

In [ ]:
# the function to classify tweets from the finetuned model and get the evaluation results
def classify_tweets(df, text_col, label_col, model, save_path="test_results",save_results=True):
    df[text_col]=df[text_col].astype(str)
    df[label_col]=df[label_col].astype(str)
    device = 0 if torch.cuda.is_available() else -1  # Use GPU if available
    classifier = pipeline(
        "text-classification",
        model=model,
        tokenizer=model,
        device=device,
        truncation=True,  
        max_length=512,   
        padding="max_length"  
    )
    
    outcomes, probs, pred_labels = [], [], []
    for text in tqdm_notebook(df[text_col]):  
        preds = classifier(text, return_all_scores=True)
        outcomes.append(preds)
        
        # Extract probabilities and predicted label
        label_scores = {entry['label']: entry['score'] for entry in preds[0]}
        probs.append(list(label_scores.values()))
        pred_labels.append(max(label_scores, key=label_scores.get))
    df["predicted_label"] = pred_labels
    
    # Compute evaluation metrics
    labels = df[label_col].tolist()
    precision, recall, f1, _ = precision_recall_fscore_support(labels, pred_labels, average="weighted")
    acc = accuracy_score(labels, pred_labels)
    
    # Ensure labels are correctly one-hot encoded for AUROC & AUPRC
    label_dummies = pd.get_dummies(labels).reindex(columns=label_scores.keys(), fill_value=0)
    
    auroc = roc_auc_score(label_dummies, probs, average="weighted", multi_class="ovr")  
    auprc = average_precision_score(label_dummies, probs, average="weighted")
    
    results = {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'auroc': auroc,
        'auprc': auprc
    }
    print(results)
    # Save results
    if save_results==True:
        metrics_path = f"{save_path}/metrics/{model}.json"
        os.makedirs(os.path.dirname(metrics_path), exist_ok=True)
        with open(metrics_path, "w") as f:
            json.dump(results, f, indent=4)

        print("Results saved at:", metrics_path)
        roc_prc_results = {
            "roc_auc_scores": auroc,
            "prc_auc_scores": auprc
        }
        try:
            # Save ROC & PRC values
            roc_prc_path = f"{save_path}/roc_and_prc_curves/{model}.json"
            os.makedirs(os.path.dirname(roc_prc_path), exist_ok=True)
            with open(roc_prc_path, "w") as f:
                json.dump(roc_prc_results, f, indent=4)
            print("ROC & PRC scores saved at:", roc_prc_path)
        except Exception as e:
            print(f"Error saving ROC & PRC scores at {roc_prc_path}: {e}")
    
    # Clear memory
    del classifier
    gc.collect()
    torch.cuda.empty_cache()


# Part I: Selecting from continous time-series clustering and finetuning a single unified model. 

We first show how we sample from continous time series clustering and finetune to a single unified model (no merging)

In [ ]:
# read dataset
df_climate=pd.read_csv("../Data/Climate_change/climate_change_cleaned_data.csv.gz", compression="gzip", index_col=False)
df_climate["created_at"]=pd.to_datetime(df_climate["created_at"])
df_climate['sentiment'] = df_climate['sentiment'].replace({2:"News",1:"Pro",0:"Neutral",-1:"Anti"})

# perform continous time series clustering and selecting based on the clusters
df_climate=continous_time_series_clustering(df=df_climate,
                                            time_col="created_at",
                                            level="M", # NOTE: level refers to the level of granularity of which we wish to cluster: "M" here stands for Month, "YE" stands for year, etc. 
                                            plot=False, penalty=0.1) 
df_climate=cluster_sampling(df=df_climate,sample_size=10000,stratified_col="cluster") #note: stratified co
df_climate.to_csv("../Data/Climate_change/temp/climate_data_cluster_month.csv.gz", compression="gzip", index=False)

df_climate_filter=df_climate[df_climate["selected"]==1].reset_index(drop=True)
for col in df_climate_filter.columns:
    if isinstance(df_climate_filter[col].dtype, pd.PeriodDtype):
        df_climate_filter[col] = df_climate_filter[col].astype(str)
        

In [ ]:
# finetuning
finetune(df=df_climate_filter, model_name="microsoft/deberta-v3-large",cluster_col_name=None,
         folder_name="naive_finetuning/continous_clustering_by_month/climate_change",
         text_col='message', label_col="sentiment",  label2id={"Anti":0,"Neutral":1,"Pro":2,"News":3},
         learning_rate=1e-5, warmup_ratio=0.05, weight_decay=0.001,
         epochs=3, batch_size=6, early_stopping_patience=2, return_val_data=False)

gc.collect()
torch.cuda.empty_cache()

Finetuning!


C:\Users\charl\anaconda3\envs\pytorch_env\lib\site-packages\transformers\models\auto\tokenization_auto.py:823: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
C:\Users\charl\anaconda3\envs\pytorch_env\lib\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
C:\Users\charl\anaconda3\envs\pytorch_env\lib\site-packages\transformers\models\auto\auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `toke

Map:   0%|          | 0/9001 [00:00<?, ? examples/s]

Map:   0%|          | 0/1001 [00:00<?, ? examples/s]

C:\Users\charl\anaconda3\envs\pytorch_env\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\charl\OneDrive - Washington University in St. Louis\Research\Dissertation\temporal_sensitive_sentiment_analysis\Code\fine_tuning.py:110: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Auroc,Auprc
1,0.766200,0.579091,0.787213,0.780955,0.784476,0.787213,0.934573,0.869537
2,0.458100,0.744201,0.827173,0.821763,0.825826,0.827173,0.943314,0.892887
3,0.255800,0.902342,0.827173,0.823738,0.824093,0.827173,0.944141,0.892331


Deleted: ./results/naive_finetuning/continous_clustering_by_month/climate_change


In [12]:
# evaluation!
df_climate=pd.read_csv("../Data/Climate_change/temp/climate_data_cluster_month.csv.gz", compression="gzip", index_col=False)
df_climate=df_climate[df_climate["selected"]==0].reset_index(drop=True)
classify_tweets(df_climate, "message", "sentiment", 
                model='models/naive_finetuning/continous_clustering_by_month/climate_change/deberta-v3-large/0', 
                save_path="test_results",
                save_results=False)



Device set to use cuda:0


  0%|          | 0/33941 [00:00<?, ?it/s]

C:\Users\charl\anaconda3\envs\pytorch_env\lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


{'accuracy': 0.8178898677116172, 'f1': 0.8148961393780465, 'precision': 0.8141299746319381, 'recall': 0.8178898677116172, 'auroc': 0.9404712417983264, 'auprc': 0.885129044604429}


# Part II: Finetuning each model to a cluster, and merging these models. 

Here we demonstrate how we:
1. Finetune each model to each identified cluster
2. Merge these multiple models using `greedy souping` and `TIES` (we demo these as they are the most competitive). 



In [ ]:
# Finetune multiple models, each for one cluster.    
df_val=finetune(df=df_climate_filter, model_name="microsoft/deberta-v3-large",
                cluster_col_name="cluster", # the difference is here --> we use "cluster" instead of none to indicate we want to cluster multiple models based on the "cluster" column
                folder_name="batch_finetuning/continous_clustering_by_month/climate_change", # the individual models will all be saved here
                text_col='message', 
                label_col="sentiment",  
                label2id={"Anti":0,"Neutral":1,"Pro":2,"News":3},
                learning_rate=1e-5, 
                warmup_ratio=0.05, 
                weight_decay=0.001,
                epochs=8, 
                batch_size=6, 
                early_stopping_patience=2, 
                return_val_data=True)

gc.collect()
torch.cuda.empty_cache()

  0%|          | 0/6 [00:00<?, ?it/s]

Finetuning!


C:\Users\charl\anaconda3\envs\pytorch_env\lib\site-packages\transformers\models\auto\tokenization_auto.py:823: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
C:\Users\charl\anaconda3\envs\pytorch_env\lib\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
C:\Users\charl\anaconda3\envs\pytorch_env\lib\site-packages\transformers\models\auto\auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `toke

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/167 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/167 [00:00<?, ? examples/s]

C:\Users\charl\anaconda3\envs\pytorch_env\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\charl\OneDrive - Washington University in St. Louis\Research\Dissertation\temporal_sensitive_sentiment_analysis\Code\fine_tuning.py:110: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Auroc,Auprc
1,1.170800,0.935695,0.622754,0.601272,0.665724,0.622754,0.820740,0.696387
2,0.718600,0.903063,0.694611,0.692676,0.712390,0.694611,0.862554,0.749116
3,0.454000,1.201689,0.688623,0.677395,0.694791,0.688623,0.865439,0.763749
4,0.218000,1.551955,0.736527,0.726300,0.746840,0.736527,0.863797,0.770118
5,0.139300,1.884521,0.724551,0.715193,0.729573,0.724551,0.855765,0.763548


Deleted: ./results/batch_finetuning/continous_clustering_by_month/climate_change
Deleted: ./results/batch_finetuning/continous_clustering_by_month/climate_change


#### Greedy souping

In [ ]:
# Merge the models 
label2id={"Anti":0,"Neutral":1,"Pro":2,"News":3}
num_labels=4
text_col="message"
col_label="sentiment"

In [ ]:
# we perform greedy souping
greedy_souping(
    models_folder="models/batch_finetuning/continous_clustering_by_month/climate_change/deberta-v3-large", # where the models are saved
    save_path="models/merged_models/continous_clustering_by_month/climate_change/greedy_soup/deberta-v3-large", # where you wish to save your merged model
    df_val=df_val,          # your validation dataset
    col_label=col_label,
    text_col=text_col,
    num_labels=num_labels,
    label2id=label2id)

Scoring checkpoints on validation set …


  0%|          | 0/6 [00:00<?, ?it/s]

C:\Users\charl\OneDrive - Washington University in St. Louis\Research\Dissertation\temporal_sensitive_sentiment_analysis\Code\merging.py:139: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=fp16_ok, dtype=torch.float16):
C:\Users\charl\OneDrive - Washington University in St. Louis\Research\Dissertation\temporal_sensitive_sentiment_analysis\Code\merging.py:139: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=fp16_ok, dtype=torch.float16):
C:\Users\charl\OneDrive - Washington University in St. Louis\Research\Dissertation\temporal_sensitive_sentiment_analysis\Code\merging.py:139: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=fp16_ok, dtype=torch.float

Start soup with models/batch_finetuning/continous_clustering_by_month/climate_change/deberta-v3-large\0  (F1 = 0.7334)


  0%|          | 0/5 [00:00<?, ?it/s]

...

skipped models/batch_finetuning/continous_clustering_by_month/climate_change/deberta-v3-large\1   (F1 = 0.7372)


C:\Users\charl\OneDrive - Washington University in St. Louis\Research\Dissertation\temporal_sensitive_sentiment_analysis\Code\merging.py:139: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=fp16_ok, dtype=torch.float16):


skipped models/batch_finetuning/continous_clustering_by_month/climate_change/deberta-v3-large\4   (F1 = 0.7262)


C:\Users\charl\OneDrive - Washington University in St. Louis\Research\Dissertation\temporal_sensitive_sentiment_analysis\Code\merging.py:139: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=fp16_ok, dtype=torch.float16):


skipped models/batch_finetuning/continous_clustering_by_month/climate_change/deberta-v3-large\3   (F1 = 0.6408)

Greedy soup (2 checkpoints) saved to models/merged_models/continous_clustering_by_month/climate_change/greedy_soup/deberta-v3-large  |  dev F1 = 0.7465


In [ ]:
results, df=classify_tweets(df_climate, text_col, col_label, 
                            model="models/merged_models/continous_clustering_by_month/climate_change/greedy_soup/deberta-v3-large", 
                            save_path="test_results",
                            save_results=True)

Device set to use cuda:0


  0%|          | 0/33941 [00:00<?, ?it/s]

C:\Users\charl\anaconda3\envs\pytorch_env\lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


Results saved at: test_results/metrics/models/merged_models/continous_clustering_by_month/climate_change/greedy_soup/deberta-v3-large.json
{'accuracy': 0.78660027695118, 'f1': 0.7840331282744417, 'precision': 0.783088115851741, 'recall': 0.78660027695118, 'auroc': 0.9161061382252383, 'auprc': 0.8479873908663575}
ROC & PRC scores saved at: test_results/roc_and_prc_curves/models/merged_models/continous_clustering_by_month/climate_change/greedy_soup/deberta-v3-large.json


#### TIES merging

In [ ]:
# First we select the optimal parameters from the held-out validation set
top_ks=[10,20,30]
lambda_scales=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
deberta_save_paths, deberta_lambda_scales,deberta_top_ks,deberta_results,deberta_f_score=[],[],[],[],[]
for top_k in tqdm_notebook(top_ks):
    for lambda_scale in lambda_scales:
        lambda_scale_str = str(lambda_scale).replace('.', '_') 
        save_path=f"models/merged_models/continous_clustering_by_month/climate_change/ties/deberta-v3-large_{top_k}_{lambda_scale_str}"
        ties(models_folder="models/batch_finetuning/continous_clustering_by_month/climate_change/deberta-v3-large", 
             save_path=save_path, base_model_name="microsoft/deberta-v3-large",
             num_labels=4,label2id=label2id,top_k=top_k,lambda_scale=lambda_scale)
        
        results, df=classify_tweets(df_val, 'message', "sentiment",   
                                    model=save_path, 
                                    save_path="test_results",
                                    save_results=False)
        saved_path=f"test_results/metrics/{save_path}.json"  
        deberta_save_paths.append(save_path)
        deberta_lambda_scales.append(lambda_scale)
        deberta_top_ks.append(top_k)
        deberta_results.append(results)
        f_score=float(results["f1"])
        deberta_f_score.append(f_score)

  0%|          | 0/3 [00:00<?, ?it/s]

C:\Users\charl\anaconda3\envs\pytorch_env\lib\site-packages\transformers\models\auto\auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading fine-tuned models:   0%|          | 0/6 [00:00<?, ?it/s]

Global top-k% trimming:   0%|          | 0/6 [00:00<?, ?it/s]

Elect sign & merge:   0%|          | 0/393 [00:00<?, ?it/s]

...

TIES-merged model saved to models/merged_models/continous_clustering_by_month/climate_change/ties/bert-large-uncased_30_1.0


Device set to use cuda:0


  0%|          | 0/1002 [00:00<?, ?it/s]

C:\Users\charl\anaconda3\envs\pytorch_env\lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


Results saved at: test_results/metrics/models/merged_models/continous_clustering_by_month/climate_change/ties/bert-large-uncased_30_1.0.json
{'accuracy': 0.6936127744510978, 'f1': 0.6853148855568153, 'precision': 0.6987170959017588, 'recall': 0.6936127744510978, 'auroc': 0.856299113260668, 'auprc': 0.7366383410951107}
ROC & PRC scores saved at: test_results/roc_and_prc_curves/models/merged_models/continous_clustering_by_month/climate_change/ties/bert-large-uncased_30_1.0.json


In [ ]:
# printing out the optimal parameters and the validation results
best_index = deberta_f_score.index(max(deberta_f_score))
best_lambda=deberta_lambda_scales[best_index]
best_top_k=deberta_top_ks[best_index]
best_results=deberta_results[best_index]
deberta_best_save_path=deberta_save_paths[best_index]
print_statement=f'''
Best lambda:{best_lambda}
Best top_k:{best_top_k}
Best results:{best_results}
'''

print(print_statement)


Best lambda:0.6
Best top_k:10
Best results:{'accuracy': 0.7544910179640718, 'f1': 0.7413933415380723, 'precision': 0.7561588564437421, 'recall': 0.7544910179640718, 'auroc': 0.913731013267178, 'auprc': 0.8383277881629235}



In [ ]:
# evaluate!
# Now we run the model with the optimal parameters. 
# NOTE: we have saved the models earlier, so we are simply calling and evaluating the 'best' performing model here. 
results, df=classify_tweets(df_climate, 'message', "sentiment",
                            model=deberta_best_save_path, 
                            save_path="test_results")

Device set to use cuda:0


  0%|          | 0/33941 [00:00<?, ?it/s]

C:\Users\charl\anaconda3\envs\pytorch_env\lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


Results saved at: test_results/metrics/models/merged_models/continous_clustering_by_month/climate_change/ties/deberta-v3-large_10_0_6.json
{'accuracy': 0.768657376034884, 'f1': 0.7550443343741022, 'precision': 0.7667735762042582, 'recall': 0.768657376034884, 'auroc': 0.9150627671642206, 'auprc': 0.841546060804306}
ROC & PRC scores saved at: test_results/roc_and_prc_curves/models/merged_models/continous_clustering_by_month/climate_change/ties/deberta-v3-large_10_0_6.json
